### Importar bibliotecas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time

### Inicializar o navegador

In [ ]:
servico = Service(ChromeDriverManager().install())
nav = webdriver.Chrome(service=servico)

### Importar/ visualizar a base de dados

In [ ]:
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

### Definir as funções de busca do Google Shopping e do Buscapé

In [ ]:
# função para analisar se o nome do produto não tem nenhum termo banido
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

# função para analisar se o nome do produto tem todos os termos do produto
def verificar_tem_todos_termos_produto(lista_termos_produto, nome):
    tem_todos_termos_produto = True
    for palavra in lista_termos_produto:
        if palavra not in nome:
            tem_todos_termos_produto = False
    return tem_todos_termos_produto

# função de busca do Google Shopping
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    #tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo) 

    # buscar pelo produto no Google
    nav.get("https://www.google.com.br/")
    nav.find_element(By.CLASS_NAME,'gLFyf').send_keys(produto, Keys.ENTER)

    # entrar na aba Shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # pegar as informações (nome, preco, link) do produto na aba Shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'i0X6df')

    for resultado in lista_resultados: 
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()
        
        # analisar se o nome do produto não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se o nome do produto tem todos os termos do produto
        tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_produto, nome)

        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_termos_produtos = True
        if not tem_termos_banidos and tem_todos_termos_produto:
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace("R$", "").replace(" ", "").replace(".","").replace(",",".")
            preco = float(preco)

            # verificar se o preço do produto está entre o preço mínimo e o preço máximo
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element(By.CLASS_NAME, 'bONr3b')
                elemento_pai = elemento_referencia.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute ('href')
                lista_ofertas.append((nome, preco, link))
                
    # retornar a lista de ofertas do Google Shopping
    return lista_ofertas

# função de busca do Buscapé
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    lista_ofertas = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo) 
    
    # buscar pelo produto no Buscapé
    nav.get("https://www.buscape.com.br/")
    nav.find_element(By.XPATH,'//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    # pegar as informações (nome, preco, link) do produto no Buscapé
    while len(nav.find_elements(By.CLASS_NAME, 'Select_Select__1S7HV')) < 1:
        time.sleep(1)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    
    for resultado in lista_resultados:
        preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
        nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
        nome = nome.lower()
        link = resultado.get_attribute("href")

        # analisar se o nome do produto não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se o nome do produto tem todos os termos do produto
        tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_produto, nome)
    
        #selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_termos_produtos = True
        if not tem_termos_banidos and tem_todos_termos_produto:
            preco = preco.replace("R$", "").replace(" ", "").replace(".","").replace(",",".")
            preco = float(preco)
            
             # verificar se o preço do produto está entre o preço mínimo e o preço máximo
            if preco_minimo <= preco <= preco_maximo:
                lista_ofertas.append((nome, preco, link))
            
    # retornar a lista de ofertas do Buscapé
    return lista_ofertas

### Salvar as ofertas desejadas em um dataframe

In [ ]:
tabela_oferta = pd.DataFrame()

for linha in tabela_produtos.index:
    # pesquisar pelo produto
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    # executar as funções do Google Shopping e do Buscapé
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shoppping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preço', 'link'])
        tabela_oferta = pd.concat([tabela_oferta, tabela_google_shoppping])
    else:
        tabela_google_shoppping = None
        
    lista_ofertas_buscapé = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscapé:
        tabela_buscape =pd.DataFrame(lista_ofertas_buscapé, columns=['produto', 'preço', 'link'])
        tabela_oferta = pd.concat([tabela_oferta, tabela_buscape])
    else:
        tabela_buscape = None
        
display(tabela_oferta)  

### Exportar para o excel o resultado do dataframe

In [ ]:
tabela_oferta = tabela_oferta.reset_index(drop=True)
tabela_oferta.to_excel("Ofertas.xlsx", index=False)

### Enviar por e-mail o resultado do dataframe

In [ ]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

# verificar se existe alguma oferta dentro da tabela de ofertas
if len(tabela_oferta.index) > 0:
    # enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'inserir o e-mail do destinatário'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_oferta.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  